In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score, train_test_split
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import warnings
import clean
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
rawdf = pd.read_csv('train.csv')

In [4]:
rawdf.columns = [col.replace(' ', '_').lower() for col in rawdf.columns]

In [5]:
for col in rawdf.columns:
    print(col)

id
pid
ms_subclass
ms_zoning
lot_frontage
lot_area
street
alley
lot_shape
land_contour
utilities
lot_config
land_slope
neighborhood
condition_1
condition_2
bldg_type
house_style
overall_qual
overall_cond
year_built
year_remod/add
roof_style
roof_matl
exterior_1st
exterior_2nd
mas_vnr_type
mas_vnr_area
exter_qual
exter_cond
foundation
bsmt_qual
bsmt_cond
bsmt_exposure
bsmtfin_type_1
bsmtfin_sf_1
bsmtfin_type_2
bsmtfin_sf_2
bsmt_unf_sf
total_bsmt_sf
heating
heating_qc
central_air
electrical
1st_flr_sf
2nd_flr_sf
low_qual_fin_sf
gr_liv_area
bsmt_full_bath
bsmt_half_bath
full_bath
half_bath
bedroom_abvgr
kitchen_abvgr
kitchen_qual
totrms_abvgrd
functional
fireplaces
fireplace_qu
garage_type
garage_yr_blt
garage_finish
garage_cars
garage_area
garage_qual
garage_cond
paved_drive
wood_deck_sf
open_porch_sf
enclosed_porch
3ssn_porch
screen_porch
pool_area
pool_qc
fence
misc_feature
misc_val
mo_sold
yr_sold
sale_type
saleprice


### Deal with Missing Data

#### Columns with Explained NA 

BsmtQual: Height of the basement  
NA No Basement

BsmtCond: General condition of the basement  
NA No Basement

BsmtFinType1: Quality of basement finished area  
NA No Basement

BsmtFinType2: Quality of second finished area (if present)  
NA No Basement

FireplaceQu: Fireplace quality  
NA No Fireplace

GarageType: Garage location  
NA No Garage

GarageFinish: Interior finish of the garage  
NA No Garage

GarageQual: Garage quality  
NA No Garage

GarageCond: Garage condition  
NA No Garage

PoolQC: Pool quality  
NA No Pool

Fence: Fence quality  
NA No Fence

MiscFeature: Miscellaneous feature not covered in other categories  
NA None


In [6]:
ng = clean.null_groups(rawdf)
for k,v in ng.items():
    print(k,v)

22 ['mas_vnr_type', 'mas_vnr_area']
55 ['bsmt_qual', 'bsmt_cond', 'bsmtfin_type_1']
2 ['bsmt_full_bath', 'bsmt_half_bath']
114 ['garage_yr_blt', 'garage_finish', 'garage_qual', 'garage_cond']


The basement group with 55 NaN values are all explained.  
The garage group with 114 NaN values are all explained, but garage_yr_built must be dropped.  
The garage condition and garage quality ratings capture the value of the garage.  
Must investigate the null values of masonry group and the other basement group.

In [7]:
rawdf['mas_vnr_type'].value_counts(dropna=False)

None       1218
BrkFace     630
Stone       168
NaN          22
BrkCmn       13
Name: mas_vnr_type, dtype: int64

In [8]:
rawdf['mas_vnr_area'].value_counts(dropna=False).head()

 0.0      1216
NaN         22
 120.0      11
 200.0      10
 176.0      10
Name: mas_vnr_area, dtype: int64

Masonry column mas_vnr_type contains both None and NaN.  
Masonry column mas_vnr_area contains both None and NaN.  
There are no other masonry columns from which to logically impute the NaN values.
These rows will need to be dropped or masonry columns will need to be dropped.

#### Check the remaining columns with explained null values. Fireplace, pool, misc, and fence.

In [9]:
rawdf['pool_qc'].value_counts(dropna=False)

NaN    2042
Gd        4
TA        2
Fa        2
Ex        1
Name: pool_qc, dtype: int64

In [10]:
rawdf['pool_area'].value_counts(dropna=False)

0      2042
561       1
519       1
800       1
738       1
648       1
576       1
480       1
368       1
228       1
Name: pool_area, dtype: int64

It can be seen that all pool quality NaN values correspond to pool area = 0.  
Pool quality NaN values indicate no pool.  

In [11]:
(rawdf['pool_qc'].isnull() & rawdf['pool_area']!=0.0).any() # False

False

The fireplace quality NaN values are all explained. 

In [12]:
rawdf['fireplace_qu'].value_counts(dropna=False)

NaN    1000
Gd      523
TA      407
Fa       59
Po       31
Ex       31
Name: fireplace_qu, dtype: int64

The fence quality NaN values are all explained. 

In [13]:
rawdf['fence'].value_counts(dropna=False)

NaN      1651
MnPrv     227
GdPrv      83
GdWo       80
MnWw       10
Name: fence, dtype: int64

Miscellaneous Features and Values have explained NaN values.

In [14]:
rawdf['misc_feature'].value_counts(dropna=False)

NaN     1986
Shed      56
Gar2       4
Othr       3
Elev       1
TenC       1
Name: misc_feature, dtype: int64

In [15]:
(rawdf['misc_feature'].isnull() & rawdf['misc_val']!=0.0).any()

False

In [16]:
rawdf['garage_type'].value_counts(dropna=False)

Attchd     1213
Detchd      536
BuiltIn     132
NaN         113
Basment      27
2Types       19
CarPort      11
Name: garage_type, dtype: int64

In [17]:
rawdf[rawdf['garage_type'].notnull() & rawdf['garage_qual'].isnull()]
# conflicting info about garage 6:1 columns say there is no garage  
# impute garage type to NaN 

id        pid  ms_subclass ms_zoning  lot_frontage  lot_area street  \
1712  2237  910201180           70        RM          50.0      9060   Pave   

     alley lot_shape land_contour utilities lot_config land_slope  \
1712   NaN       Reg          Lvl    AllPub     Inside        Gtl   

     neighborhood condition_1 condition_2 bldg_type house_style  overall_qual  \
1712       IDOTRR        Norm        Norm      1Fam      2Story             5   

      overall_cond  year_built  year_remod/add roof_style roof_matl  \
1712             6        1923            1999      Gable   CompShg   

     exterior_1st exterior_2nd mas_vnr_type  mas_vnr_area exter_qual  \
1712      Wd Sdng      Plywood         None           0.0         TA   

     exter_cond foundation bsmt_qual bsmt_cond bsmt_exposure bsmtfin_type_1  \
1712         TA     BrkTil        Gd        TA            No            ALQ   

      bsmtfin_sf_1 bsmtfin_type_2  bsmtfin_sf_2  bsmt_unf_sf  total_bsmt_sf  \
1712         548.0            Unf           0.0        311.0          859.0   

     heating heating_qc central_air electrical  1st_flr_sf  2nd_flr_sf  \
1712    GasA         Ex           Y      SBrkr         942         886   

      low_qual_fin_sf  gr_liv_area  bsmt_full_bath  bsmt_half_bath  full_bath  \
1712                0         1828             0.0             0.0          2   

      half_bath  bedroom_abvgr  kitchen_abvgr kitchen_qual  totrms_abvgrd  \
1712          0              3              1           Gd              6   

     functional  fireplaces fireplace_qu garage_type  garage_yr_blt  \
1712        Typ           0          NaN      Detchd            NaN   

     garage_finish  garage_cars  garage_area garage_qual garage_cond  \
1712           NaN          NaN          NaN         NaN         NaN   

     paved_drive  wood_deck_sf  open_porch_sf  enclosed_porch  3ssn_porch  \
1712           Y           174              0             212           0   

      screen_porch  pool_area pool_qc  fence misc_feature  misc_val  mo_sold  \
1712             0          0     NaN  MnPrv          NaN         0        3   

      yr_sold sale_type  saleprice  
1712     2007       WD      150909

In [18]:
rawdf['garage_type'][1712] = np.nan

In [19]:
rawdf.isnull().sum()[:10]

id                 0
pid                0
ms_subclass        0
ms_zoning          0
lot_frontage     330
lot_area           0
street             0
alley           1911
lot_shape          0
land_contour       0
dtype: int64

#### Lastly examine Lot Frontage and Alley  

LotFrontage: Linear feet of street connected to property

Alley: Type of alley access to property
Grvl Gravel
Pave Paved
NA No alley access

All alley NaN are explained.

Lot Frontage NaN seems to be missing at random, is not highly correlated with the target,  
and 16% of values are missing; therefore drop lot_frontage.

In [20]:
rawdf[['lot_frontage', 'saleprice']].dropna(how='any').corr()

lot_frontage  saleprice
lot_frontage      1.000000   0.341842
saleprice         0.341842   1.000000

In [21]:
rawdf.select_dtypes(exclude='object').isnull().sum()

id                   0
pid                  0
ms_subclass          0
lot_frontage       330
lot_area             0
overall_qual         0
overall_cond         0
year_built           0
year_remod/add       0
mas_vnr_area        22
bsmtfin_sf_1         1
bsmtfin_sf_2         1
bsmt_unf_sf          1
total_bsmt_sf        1
1st_flr_sf           0
2nd_flr_sf           0
low_qual_fin_sf      0
gr_liv_area          0
bsmt_full_bath       2
bsmt_half_bath       2
full_bath            0
half_bath            0
bedroom_abvgr        0
kitchen_abvgr        0
totrms_abvgrd        0
fireplaces           0
garage_yr_blt      114
garage_cars          1
garage_area          1
wood_deck_sf         0
open_porch_sf        0
enclosed_porch       0
3ssn_porch           0
screen_porch         0
pool_area            0
misc_val             0
mo_sold              0
yr_sold              0
saleprice            0
dtype: int64

In [22]:
rawdf.select_dtypes(include='object').isnull().sum()

ms_zoning            0
street               0
alley             1911
lot_shape            0
land_contour         0
utilities            0
lot_config           0
land_slope           0
neighborhood         0
condition_1          0
condition_2          0
bldg_type            0
house_style          0
roof_style           0
roof_matl            0
exterior_1st         0
exterior_2nd         0
mas_vnr_type        22
exter_qual           0
exter_cond           0
foundation           0
bsmt_qual           55
bsmt_cond           55
bsmt_exposure       58
bsmtfin_type_1      55
bsmtfin_type_2      56
heating              0
heating_qc           0
central_air          0
electrical           0
kitchen_qual         0
functional           0
fireplace_qu      1000
garage_type        114
garage_finish      114
garage_qual        114
garage_cond        114
paved_drive          0
pool_qc           2042
fence             1651
misc_feature      1986
sale_type            0
dtype: int64

Handle remaining nulls.

In [23]:
mask = (rawdf[[col for col in rawdf.columns if 'bsmt' in col]].isnull().sum(axis=1) > 0)

In [24]:
rawdf[mask][[col for col in rawdf.columns if 'bsmt' in col]]

bsmt_qual bsmt_cond bsmt_exposure bsmtfin_type_1  bsmtfin_sf_1  \
12         NaN       NaN           NaN            NaN           0.0   
93         NaN       NaN           NaN            NaN           0.0   
114        NaN       NaN           NaN            NaN           0.0   
146        NaN       NaN           NaN            NaN           0.0   
183        NaN       NaN           NaN            NaN           0.0   
240        NaN       NaN           NaN            NaN           0.0   
249        NaN       NaN           NaN            NaN           0.0   
256        NaN       NaN           NaN            NaN           0.0   
390        NaN       NaN           NaN            NaN           0.0   
437        NaN       NaN           NaN            NaN           0.0   
485        NaN       NaN           NaN            NaN           0.0   
499        NaN       NaN           NaN            NaN           0.0   
514        NaN       NaN           NaN            NaN           0.0   
580        NaN       NaN           NaN            NaN           0.0   
581        NaN       NaN           NaN            NaN           0.0   
616        NaN       NaN           NaN            NaN           0.0   
635        NaN       NaN           NaN            NaN           0.0   
642        NaN       NaN           NaN            NaN           0.0   
696        NaN       NaN           NaN            NaN           0.0   
700        NaN       NaN           NaN            NaN           0.0   
737        NaN       NaN           NaN            NaN           0.0   
772        NaN       NaN           NaN            NaN           0.0   
790        NaN       NaN           NaN            NaN           0.0   
794        NaN       NaN           NaN            NaN           0.0   
924        NaN       NaN           NaN            NaN           0.0   
933        NaN       NaN           NaN            NaN           0.0   
954        NaN       NaN           NaN            NaN           0.0   
1007       NaN       NaN           NaN            NaN           0.0   
1022       NaN       NaN           NaN            NaN           0.0   
1049       NaN       NaN           NaN            NaN           0.0   
1089       NaN       NaN           NaN            NaN           0.0   
1098       NaN       NaN           NaN            NaN           0.0   
1125       NaN       NaN           NaN            NaN           0.0   
1147        Gd        TA            No            GLQ        1124.0   
1157       NaN       NaN           NaN            NaN           0.0   
1211       NaN       NaN           NaN            NaN           0.0   
1222       NaN       NaN           NaN            NaN           0.0   
1252       NaN       NaN           NaN            NaN           0.0   
1256       NaN       NaN           NaN            NaN           0.0   
1270       NaN       NaN           NaN            NaN           0.0   
1327       NaN       NaN           NaN            NaN           NaN   
1331       NaN       NaN           NaN            NaN           0.0   
1377       NaN       NaN           NaN            NaN           0.0   
1387       NaN       NaN           NaN            NaN           0.0   
1428       NaN       NaN           NaN            NaN           0.0   
1456        Gd        TA           NaN            Unf           0.0   
1481       NaN       NaN           NaN            NaN           0.0   
1547        Gd        TA           NaN            Unf           0.0   
1594       NaN       NaN           NaN            NaN           0.0   
1620       NaN       NaN           NaN            NaN           0.0   
1681       NaN       NaN           NaN            NaN           0.0   
1684       NaN       NaN           NaN            NaN           0.0   
1686       NaN       NaN           NaN            NaN           0.0   
1859       NaN       NaN           NaN            NaN           0.0   
1875       NaN       NaN           NaN            NaN           0.0   
1889       NaN       NaN     

index 616 bsmt_full_bath, bsmt_half_bath impute to 0.0  
  
index 1327 bsmtfin_sf_1, bsmtfin_sf_2, bsmt_unf_sf, total_bsmt_sf, bsmt_full_bath, bsmt_half_bath impute to 0.0

In [25]:
for col in ['bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath']:
    rawdf[col].fillna(0.0, axis=0, inplace=True)

In [26]:
rawdf.loc[[616, 1327], [col for col in rawdf.columns if 'bsmt' in col]]

bsmt_qual bsmt_cond bsmt_exposure bsmtfin_type_1  bsmtfin_sf_1  \
616        NaN       NaN           NaN            NaN           0.0   
1327       NaN       NaN           NaN            NaN           0.0   

     bsmtfin_type_2  bsmtfin_sf_2  bsmt_unf_sf  total_bsmt_sf  bsmt_full_bath  \
616             NaN           0.0          0.0            0.0             0.0   
1327            NaN           0.0          0.0            0.0             0.0   

      bsmt_half_bath  
616              0.0  
1327             0.0

In [27]:
mask = (rawdf[[col for col in rawdf.columns if 'garag' in col]].isnull().sum(axis=1) > 0)
rawdf[mask][[col for col in rawdf.columns if 'garag' in col]]

garage_type  garage_yr_blt garage_finish  garage_cars  garage_area  \
28           NaN            NaN           NaN          0.0          0.0   
53           NaN            NaN           NaN          0.0          0.0   
65           NaN            NaN           NaN          0.0          0.0   
79           NaN            NaN           NaN          0.0          0.0   
101          NaN            NaN           NaN          0.0          0.0   
103          NaN            NaN           NaN          0.0          0.0   
114          NaN            NaN           NaN          0.0          0.0   
120          NaN            NaN           NaN          0.0          0.0   
134          NaN            NaN           NaN          0.0          0.0   
136          NaN            NaN           NaN          0.0          0.0   
139          NaN            NaN           NaN          0.0          0.0   
162          NaN            NaN           NaN          0.0          0.0   
235          NaN            NaN           NaN          0.0          0.0   
256          NaN            NaN           NaN          0.0          0.0   
260          NaN            NaN           NaN          0.0          0.0   
264          NaN            NaN           NaN          0.0          0.0   
276          NaN            NaN           NaN          0.0          0.0   
334          NaN            NaN           NaN          0.0          0.0   
356          NaN            NaN           NaN          0.0          0.0   
382          NaN            NaN           NaN          0.0          0.0   
406          NaN            NaN           NaN          0.0          0.0   
409          NaN            NaN           NaN          0.0          0.0   
426          NaN            NaN           NaN          0.0          0.0   
437          NaN            NaN           NaN          0.0          0.0   
445          NaN            NaN           NaN          0.0          0.0   
457          NaN            NaN           NaN          0.0          0.0   
486          NaN            NaN           NaN          0.0          0.0   
530          NaN            NaN           NaN          0.0          0.0   
542          NaN            NaN           NaN          0.0          0.0   
555          NaN            NaN           NaN          0.0          0.0   
...          ...            ...           ...          ...          ...   
1464         NaN            NaN           NaN          0.0          0.0   
1511         NaN            NaN           NaN          0.0          0.0   
1519         NaN            NaN           NaN          0.0          0.0   
1528         NaN            NaN           NaN          0.0          0.0   
1544         NaN            NaN           NaN          0.0          0.0   
1545         NaN            NaN           NaN          0.0          0.0   
1563         NaN            NaN           NaN          0.0          0.0   
1564         NaN            NaN           NaN          0.0          0.0   
1584         NaN            NaN           NaN          0.0          0.0   
1594         NaN            NaN           NaN          0.0          0.0   
1634         NaN            NaN           NaN          0.0          0.0   
1705         NaN            NaN           NaN          0.0          0.0   
1711         NaN            NaN           NaN          0.0          0.0   
1712         NaN            NaN           NaN          NaN          NaN   
1736         NaN            NaN           NaN          0.0          0.0   
1754         NaN            NaN           NaN          0.0          0.0   
1783         NaN            NaN           NaN          0.0          0.0   
1788         NaN            NaN           NaN          0.0          0.0   
1799         NaN            NaN           NaN          0.0          0.0   
1814         NaN            NaN           NaN          0.0          0.0   
1841         NaN            NaN           NaN          0.0          0.0   
1893         NaN            Na

In [28]:
rawdf['garage_area'].fillna(0.0, axis=0, inplace=True)

rawdf['garage_cars'].fillna(0.0, axis=0, inplace=True)

In [29]:
rawdf['mas_vnr_area'].fillna(0.0, axis=0, inplace=True)
      
rawdf['mas_vnr_type'].fillna('None', axis=0, inplace=True)

In [30]:
rawdf.drop('lot_frontage', axis=1, inplace=True)

rawdf.drop('garage_yr_blt', axis=1, inplace=True)

The remaining null values belong to categorical variables, which can be handled as dummies.

In [31]:
df = pd.get_dummies(rawdf, dummy_na=True, drop_first=True)

In [32]:
len(df['id'].unique())

2051

ID and PID are unique so PID will be dropped, it adds no info.

In [33]:
df.drop('pid', axis=1, inplace=True)

In [34]:
df.set_index('id', inplace=True, verify_integrity=True)

This is the new Data Set.

In [35]:
df.to_csv('ames_clean.csv', index=False)